In [2]:
# Orchestra Markov Chain Music Generator
# ------------------------------------------------
# • Synthesizes melodies and orchestral compositions via Markov chains
# • Supports various orchestration techniques and musical forms
# • Provides a flexible UI for experimentation in Colab/Jupyter
# ------------------------------------------------

%pip install ipywidgets --upgrade
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ModuleNotFoundError:
    pass

import ipywidgets as widgets
from IPython.display import display

import os
import random
import subprocess
import traceback
from collections import defaultdict
from typing import Dict, List, Sequence, Tuple, Optional, Union

try:
    import numpy as np
    import pretty_midi
    import matplotlib.pyplot as plt
    from ipywidgets import (
        Button, Dropdown, FloatSlider, HBox, IntSlider, Output,
        SelectMultiple, Tab, VBox, Checkbox, HTML, Label
    )
    from IPython.display import Audio, clear_output, display
    _HAS_WIDGETS = True
except ImportError:  # headless / non‑notebook environments
    _HAS_WIDGETS = False

# Helper type alias
NoteDur = Tuple[Optional[str], float]  # (pitch OR None for rest, duration in beats)

# Main generator class
class OrchestraMarkovGenerator:
    """Generate orchestral music compositions with Markov chains."""

    def __init__(self, time_signature: Tuple[int, int] = (4, 4)) -> None:
        """Initialize the generator with default settings.

        Args:
            time_signature: Tuple of (beats, beat-denominator), e.g. (4, 4) for 4/4 time
        """
        self.time_signature = time_signature

        # Basic instrument map (can be expanded)
        self.orchestra_sections = {
            "violins1": {"program": 40, "active": [True, True, False, False]},
            "violins2": {"program": 40, "active": [True, True, True, False]},
            "cellos": {"program": 42, "active": [True, True, False, True]},
            "basses": {"program": 43, "active": [True, True, False, True]},
            "flutes": {"program": 73, "active": [False, False, False, True]},
            "trumpets": {"program": 56, "active": [False, False, False, True]},
            "piano": {"program": 0, "active": [False, False, True, True]},
        }

        # Seed patterns
        self.melodic_patterns = {
            "violins1_main": [("G4", 0.5), ("A4", 0.25), ("G4", 0.25), ("C5", 1.0)],
            "cellos_bass": [("G3", 1.5), ("G3", 1.5), ("C3", 1.0)],
            "flutes_melody": [("G5", 0.25), ("A5", 0.25), ("C6", 0.5), ("G5", 1.0)],
        }

        # Default velocity map
        self.dynamic_levels = {name: 90 for name in self.orchestra_sections}

        # Build 1‑order Markov chains
        self.pitch_transitions = {}
        self.rhythm_transitions = {}
        self.build_markov_chains(order=1)

    def build_markov_chains(self, order: int = 1) -> None:
        """Build transition probability tables for pitches and rhythms.

        Args:
            order: The order of the Markov chain (default: 1)
        """
        p_counts = defaultdict(lambda: defaultdict(int))
        r_counts = defaultdict(lambda: defaultdict(int))

        for pattern in self.melodic_patterns.values():
            pitches = [p for p, _ in pattern]
            durations = [d for _, d in pattern]

            for i in range(len(pitches) - order):
                p_counts[tuple(pitches[i : i + order])][pitches[i + order]] += 1

            for i in range(len(durations) - order):
                r_counts[tuple(durations[i : i + order])][durations[i + order]] += 1

        self.pitch_transitions = {
            k: {n: c / sum(v.values()) for n, c in v.items()}
            for k, v in p_counts.items()
        }

        self.rhythm_transitions = {
            k: {n: c / sum(v.values()) for n, c in v.items()}
            for k, v in r_counts.items()
        }

    # Helper methods for common operations

    @staticmethod
    def _choice(mapping: Dict) -> Union[str, float]:
        """Select an item from mapping based on weighted probabilities.

        Args:
            mapping: Dictionary of {item: probability}

        Returns:
            Selected item
        """
        keys, probs = zip(*mapping.items())
        return random.choices(keys, probs)[0]

    def next_pitch(self, state: Tuple[str, ...], fallback: str = "C4") -> str:
        """Get the next pitch from a Markov chain state.

        Args:
            state: Current state tuple of pitches
            fallback: Default pitch if no transition exists

        Returns:
            Next pitch
        """
        return self._choice(self.pitch_transitions.get(state, {fallback: 1.0}))

    def next_duration(self, state: Tuple[float, ...], fallback: float = 0.5) -> float:
        """Get the next duration from a Markov chain state.

        Args:
            state: Current state tuple of durations
            fallback: Default duration if no transition exists

        Returns:
            Next duration value
        """
        return float(self._choice(self.rhythm_transitions.get(state, {fallback: 1.0})))

    @staticmethod
    def transpose(note: str, semitones: int) -> str:
        """Transpose a note by a number of semitones.

        Args:
            note: Note string (e.g., 'C4', 'F#3')
            semitones: Number of semitones to transpose (positive or negative)

        Returns:
            Transposed note string
        """
        names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

        # Handle notes with accidentals
        if len(note) > 2 and note[1] in ['#', 'b']:
            note_name = note[:2]
            octave = int(note[2:])
        else:
            note_name = note[0]
            octave = int(note[1:])

        # Find the index of the note name
        if note_name in names:
            idx = names.index(note_name)
        else:
            # Handle flat notes by converting to the equivalent sharp name
            if len(note_name) > 1 and note_name[1] == 'b':
                base_note = note_name[0]
                base_idx = names.index(base_note)
                idx = (base_idx - 1) % 12
            else:
                raise ValueError(f"Invalid note name: {note_name}")

        midi = idx + 12 * octave + semitones
        new_oct, new_idx = divmod(midi, 12)
        return f"{names[new_idx]}{new_oct}"

    def note_to_midi(self, note_str: str) -> int:
        """Convert a note string (e.g., 'C4') to MIDI note number.

        Args:
            note_str: Note string (e.g., 'C4', 'F#3')

        Returns:
            MIDI note number
        """
        if note_str is None:
            return 0

        # Parse the note
        name = note_str[0]

        # Handle accidentals
        accidental = 0
        idx = 1
        if idx < len(note_str) and note_str[idx] in ['#', 'b']:
            accidental = 1 if note_str[idx] == '#' else -1
            idx += 1

        # Get octave
        if idx < len(note_str) and note_str[idx].isdigit():
            octave = int(note_str[idx:])
        else:
            octave = 4  # Default octave

        # Map note name to semitone offset from C
        note_map = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
        semitone = note_map.get(name, 0) + accidental

        # Calculate MIDI note number
        return 60 + semitone + (octave - 4) * 12

    # Melody generation methods

    def generate_motif(
        self,
        length: int,
        instrument: str,
        order: int = 1,
        start_pitch_state: Optional[Tuple[str, ...]] = None,
        start_rhythm_state: Optional[Tuple[float, ...]] = None,
    ) -> List[NoteDur]:
        """Generate a short motif using Markov chains.

        Args:
            length: Desired length of the motif in notes
            instrument: Instrument name
            order: Markov chain order
            start_pitch_state: Initial pitch state (optional)
            start_rhythm_state: Initial rhythm state (optional)

        Returns:
            List of (note, duration) pairs
        """
        seed = next(iter(self.melodic_patterns.values()))

        if start_pitch_state is None:
            start_pitch_state = tuple(p for p, _ in seed[:order])

        if start_rhythm_state is None:
            start_rhythm_state = tuple(d for _, d in seed[:order])

        motif = list(zip(start_pitch_state, start_rhythm_state))

        while len(motif) < length:
            n_pitch = self.next_pitch(start_pitch_state)
            n_dur = self.next_duration(start_rhythm_state)
            motif.append((n_pitch, n_dur))

            start_pitch_state = (*start_pitch_state[1:], n_pitch) if order > 1 else (n_pitch,)
            start_rhythm_state = (*start_rhythm_state[1:], n_dur) if order > 1 else (n_dur,)

        return motif

    def apply_orchestration_technique(
        self,
        motif: List[NoteDur],
        technique: str = "unison",
        instruments: Optional[Sequence[str]] = None,
    ) -> Dict[str, List[NoteDur]]:
        """Apply orchestration techniques to a motif.

        Args:
            motif: The motif as a list of (note, duration) pairs
            technique: Orchestration technique ("unison", "octave_doubling", etc.)
            instruments: List of instruments to apply the technique to

        Returns:
            Dictionary mapping instrument names to parts
        """
        if not motif:
            return {}

        instruments = list(instruments or self.orchestra_sections)
        res = {}

        if technique == "unison":
            for instr in instruments:
                res[instr] = motif.copy()

        elif technique == "octave_doubling":
            for instr in instruments:
                res[instr] = [(self.transpose(p, -12), d) if p is not None else (None, d)
                             for p, d in motif]

        else:  # Default: copy the motif for each instrument
            for instr in instruments:
                res[instr] = motif.copy()

        return res

    def generate_standalone_melody(
        self,
        length: int = 16,
        scale: str = "major",
        key: str = "C",
        complexity: float = 0.5,
        markov_order: int = 1,
    ) -> List[NoteDur]:
        """Generate a monophonic melody.

        Args:
            length: Length in beats
            scale: Scale name ("major", "minor", "pentatonic", "blues", "chromatic")
            key: Key name ("C", "F#", "Bb", etc.)
            complexity: Complexity factor (0.0-1.0)
            markov_order: Markov chain order

        Returns:
            List of (note, duration) pairs
        """
        # Scale tables
        scales = {
            "major": [0, 2, 4, 5, 7, 9, 11],
            "minor": [0, 2, 3, 5, 7, 8, 10],
            "pentatonic": [0, 2, 4, 7, 9],
            "blues": [0, 3, 5, 6, 7, 10],
            "chromatic": list(range(12)),
        }

        # Map key names to MIDI root notes
        root_midi = {
            "C": 60, "C#": 61, "Db": 61, "D": 62, "D#": 63, "Eb": 63,
            "E": 64, "F": 65, "F#": 66, "Gb": 66, "G": 67, "G#": 68,
            "Ab": 68, "A": 69, "A#": 70, "Bb": 70, "B": 71,
        }[key]

        scale_pattern = scales.get(scale, scales["major"])
        scale_pitches = [root_midi + interval + 12 * o
                        for o in range(-1, 2)
                        for interval in scale_pattern]

        # Starting note (middle of scale range)
        melody_midi = [scale_pitches[len(scale_pitches) // 2]]

        # Define rhythm patterns based on complexity
        rhythm_patterns = [0.25, 0.5, 0.75, 1.0, 1.5, 2.0]

        if complexity < 0.3:
            rhythm_weights = [0.1, 0.4, 0.1, 0.3, 0.05, 0.05]
        elif complexity < 0.7:
            rhythm_weights = [0.2, 0.3, 0.2, 0.15, 0.1, 0.05]
        else:
            rhythm_weights = [0.3, 0.3, 0.2, 0.1, 0.05, 0.05]

        # Generate rhythm sequence
        durations = []
        total = 0.0
        while total < length:
            d = random.choices(rhythm_patterns, weights=rhythm_weights)[0]
            if total + d > length:
                d = length - total
            durations.append(d)
            total += d

        # Generate pitch sequence
        for _ in range(1, len(durations)):
            last_note = melody_midi[-1]
            probs = []

            for p in scale_pitches:
                interval = abs(p - last_note)

                if interval == 0:
                    prob = 0.1 * complexity
                elif interval <= 2:
                    prob = 0.3
                elif interval <= 4:
                    prob = 0.2
                elif interval <= 7:
                    prob = 0.1 + 0.1 * complexity
                elif interval <= 12:
                    prob = 0.05 + 0.15 * complexity
                else:
                    prob = 0.01 + 0.09 * complexity

                probs.append(prob)

            probs = np.array(probs)
            probs /= probs.sum()
            next_note = np.random.choice(scale_pitches, p=probs)

            # Occasionally add chromatic notes
            if random.random() < complexity * 0.2:
                next_note += random.choice([-1, 1])

            melody_midi.append(next_note)

        # Convert MIDI numbers to note names
        names = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]
        return [(f"{names[n % 12]}{n // 12 - 1}", d) for n, d in zip(melody_midi, durations)]

    def save_melody_to_midi(
        self,
        melody: List[NoteDur],
        file_path: str = "melody.mid",
        instrument: int = 0,
        tempo: int = 120
    ) -> str:
        """Save a melody to a MIDI file.

        Args:
            melody: List of (note, duration) pairs
            file_path: Output file path
            instrument: MIDI program number
            tempo: Tempo in BPM

        Returns:
            Path to the saved MIDI file
        """
        midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)
        inst = pretty_midi.Instrument(program=instrument)

        t = 0.0
        for note_str, duration in melody:
            # Skip rests
            if note_str is None:
                t += duration
                continue

            # Parse the note
            if len(note_str) > 1:
                note_name = note_str[0]

                # Handle accidentals
                if len(note_str) > 2 and note_str[1] in ['#', 'b']:
                    accidental = 1 if note_str[1] == '#' else -1
                    octave_pos = 2
                else:
                    accidental = 0
                    octave_pos = 1

                if len(note_str) > octave_pos and note_str[octave_pos].isdigit():
                    octave = int(note_str[octave_pos:])
                else:
                    octave = 4  # Default octave

                # Map note name to semitone offset from C
                note_map = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
                semitone = note_map.get(note_name, 0) + accidental

                # Calculate MIDI note number
                midi_note = 60 + semitone + (octave - 4) * 12

                # Create Note object
                note = pretty_midi.Note(
                    velocity=100,
                    pitch=midi_note,
                    start=t,
                    end=t + duration
                )

                # Add note to instrument
                inst.notes.append(note)

            t += duration

        # Add instrument to MIDI file
        midi.instruments.append(inst)
        midi.write(file_path)
        return file_path

    # Multi-instrument composition methods

    def generate_multi_instrument_composition(
        self,
        length: int = 16,
        key: str = "C",
        scale: str = "major",
        instruments: List[str] = ["Piano", "Violin", "Flute"],
        techniques: List[str] = ["melody", "harmony", "bass"],
        tempo: int = 120,
        complexity: float = 0.5
    ) -> Dict[str, List[NoteDur]]:
        """Generate a multi-instrument composition.

        Args:
            length: Length in beats
            key: Key name
            scale: Scale name
            instruments: List of instrument names
            techniques: List of techniques to apply to each instrument
            tempo: Tempo in BPM
            complexity: Complexity factor (0.0-1.0)

        Returns:
            Dictionary mapping instrument names to parts
        """
        # Create a dictionary to store each instrument's part
        composition = {}

        # Generate a main melody first
        main_melody = self.generate_standalone_melody(
            length=length,
            scale=scale,
            key=key,
            complexity=complexity
        )

        # Assign parts based on technique
        for i, (instrument, technique) in enumerate(zip(instruments, techniques)):
            if technique == "melody":
                # Use the main melody for melody instruments
                composition[instrument] = main_melody
            elif technique == "harmony":
                # Generate harmony based on the main melody
                harmony = []
                current_time = 0.0
                melody_idx = 0

                while current_time < length and melody_idx < len(main_melody):
                    # Get current melody note and duration
                    melody_note, melody_duration = main_melody[melody_idx]

                    if melody_note is not None:
                        # Create harmony note (typically a third or sixth)
                        try:
                            # Simple harmony - 3rd or 6th in the scale
                            interval = 4 if random.random() > 0.5 else 9  # Major third or sixth
                            if scale == "minor":
                                interval = 3 if interval == 4 else 8  # Minor third or sixth

                            harmony_note = self.transpose(melody_note, interval)
                            harmony.append((harmony_note, melody_duration))
                        except Exception:
                            # Fallback if harmony calculation fails
                            harmony.append((melody_note, melody_duration))
                    else:
                        # Keep rests
                        harmony.append((None, melody_duration))

                    current_time += melody_duration
                    melody_idx += 1

                composition[instrument] = harmony
            elif technique == "bass":
                # Generate a simpler bass line with longer notes
                bass = []
                current_time = 0.0
                melody_idx = 0

                while current_time < length:
                    # Create longer bass notes (typically 2 beats)
                    bass_duration = min(2.0, length - current_time)

                    if melody_idx < len(main_melody):
                        melody_note, _ = main_melody[melody_idx]

                        if melody_note is not None:
                            # Create bass note (root note, two octaves lower)
                            try:
                                bass_note = f"{key}2"  # Root note in bass register
                                bass.append((bass_note, bass_duration))
                            except Exception:
                                # Fallback
                                bass.append((melody_note, bass_duration))
                        else:
                            # Keep rests
                            bass.append((None, bass_duration))

                    current_time += bass_duration
                    melody_idx = min(melody_idx + 2, len(main_melody) - 1)  # Skip melody notes for longer bass notes

                composition[instrument] = bass
            else:
                # Default to melody
                composition[instrument] = main_melody

        return composition

    def save_multi_instrument_to_midi(
        self,
        composition: Dict[str, List[NoteDur]],
        file_path: str = "multi_instrument_composition.mid",
        instrument_mapping: Optional[Dict[str, int]] = None,
        tempo: int = 120
    ) -> str:
        """Save a multi-instrument composition to a MIDI file.

        Args:
            composition: Dictionary mapping instrument names to parts
            file_path: Output file path
            instrument_mapping: Dictionary mapping instrument names to MIDI program numbers
            tempo: Tempo in BPM

        Returns:
            Path to the saved MIDI file
        """
        midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)

        # Default MIDI program mapping
        default_mapping = {
            'Piano': 0, 'Violin': 40, 'Flute': 73, 'Clarinet': 71,
            'Trumpet': 56, 'Guitar': 24, 'Music Box': 10, 'Harp': 46,
            'Bass': 32, 'Cello': 42
        }

        # Use provided mapping or default
        instrument_mapping = instrument_mapping or default_mapping

        # Create a track for each instrument
        for instrument_name, notes in composition.items():
            # Get MIDI program number
            program = instrument_mapping.get(instrument_name, 0)

            # Create MIDI instrument
            midi_instrument = pretty_midi.Instrument(program=program, name=instrument_name)

            # Add notes
            time = 0.0
            for note_str, duration in notes:
                # Skip rests
                if note_str is None:
                    time += duration
                    continue

                # Parse the note
                if len(note_str) > 1:
                    try:
                        note_name = note_str[0]

                        # Handle accidentals
                        if len(note_str) > 2 and note_str[1] in ['#', 'b']:
                            accidental = 1 if note_str[1] == '#' else -1
                            octave_pos = 2
                        else:
                            accidental = 0
                            octave_pos = 1

                        if len(note_str) > octave_pos and note_str[octave_pos].isdigit():
                            octave = int(note_str[octave_pos:])
                        else:
                            octave = 4  # Default octave

                        # Map note name to semitone offset from C
                        note_map = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
                        semitone = note_map.get(note_name, 0) + accidental

                        # Calculate MIDI note number
                        midi_note = 60 + semitone + (octave - 4) * 12

                        # Create Note object
                        note = pretty_midi.Note(
                            velocity=100,
                            pitch=midi_note,
                            start=time,
                            end=time + duration
                        )

                        # Add note to instrument
                        midi_instrument.notes.append(note)
                    except Exception:
                        # Skip notes that can't be parsed
                        pass

                time += duration

            # Add instrument to MIDI file
            midi.instruments.append(midi_instrument)

        # Write the MIDI file
        midi.write(file_path)
        return file_path

    # Expressive music methods (dynamics and articulations)

    def add_dynamics(
        self,
        melody: List[NoteDur],
        shape: str = "arch"
    ) -> List[Tuple[Optional[str], float, int]]:
        """Add dynamics (volume) to a melody for more expressiveness.

        Args:
            melody: List of (note, duration) pairs
            shape: Dynamic shape ("arch", "wave", "random", "terraced")

        Returns:
            List of (note, duration, volume) tuples
        """
        import math
        result = []

        if shape == "arch":
            # Create an arch shape (quiet → loud → quiet)
            midpoint = len(melody) // 2
            for i, (note, duration) in enumerate(melody):
                # Calculate volume (0-127 MIDI range)
                if i < midpoint:
                    volume = 60 + int((i / midpoint) * 40)  # Crescendo to forte
                else:
                    volume = 100 - int(((i - midpoint) / (len(melody) - midpoint)) * 40)  # Diminuendo
                result.append((note, duration, volume))

        elif shape == "wave":
            # Create waves of dynamics
            wave_length = 8  # Length of each dynamic wave in notes
            for i, (note, duration) in enumerate(melody):
                phase = (i % wave_length) / wave_length
                volume = 75 + int(25 * math.sin(phase * 2 * math.pi))
                result.append((note, duration, volume))

        elif shape == "random":
            # Random dynamics with some coherence
            current_volume = 80
            max_change = 15
            for note, duration in melody:
                # Random walk with constraints to avoid abrupt changes
                volume_change = random.randint(-max_change, max_change)
                current_volume = max(40, min(110, current_volume + volume_change))
                result.append((note, duration, current_volume))

        elif shape == "terraced":
            # Baroque-style terraced dynamics (sudden changes)
            sections = max(2, len(melody) // 6)  # Create several sections
            for i, (note, duration) in enumerate(melody):
                section = i // (len(melody) // sections)
                # Alternate between forte and piano
                volume = 100 if section % 2 == 0 else 65
                result.append((note, duration, volume))

        else:  # default - constant dynamics with slight variations
            for note, duration in melody:
                # Add slight variations to prevent mechanical sound
                volume = 85 + random.randint(-5, 5)
                result.append((note, duration, volume))

        return result

    def add_articulations(
        self,
        melody: List[Tuple[Optional[str], float, int]],
        style: str = "mixed"
    ) -> List[Tuple[Optional[str], float, int, str]]:
        """Add articulations to a melody for more expressiveness.

        Args:
            melody: List of (note, duration, volume) tuples
            style: Articulation style ("legato", "staccato", "mixed", "phrase")

        Returns:
            List of (note, duration, volume, articulation) tuples
        """
        result = []

        if style == "legato":
            # Smooth, connected playing
            for note, duration, volume in melody:
                # Extend note duration slightly to create overlap
                modified_duration = duration * 1.05
                result.append((note, modified_duration, volume, "legato"))

        elif style == "staccato":
            # Short, detached notes
            for note, duration, volume in melody:
                # Shorten note duration
                modified_duration = duration * 0.5
                result.append((note, modified_duration, volume, "staccato"))

        elif style == "phrase":
            # Phrasing with legato within phrases, breaks between phrases
            phrase_length = 4  # notes per phrase
            for i, (note, duration, volume) in enumerate(melody):
                position_in_phrase = i % phrase_length

                if position_in_phrase < phrase_length - 1:
                    # Within phrase - legato
                    articulation = "legato"
                    modified_duration = duration * 1.05  # Slight overlap
                else:
                    # End of phrase - slight separation
                    articulation = "normal"
                    modified_duration = duration * 0.9  # Small break

                result.append((note, modified_duration, volume, articulation))

        elif style == "mixed":
            # Mix of different articulations based on musical context
            for i, (note, duration, volume) in enumerate(melody):
                # Algorithm to determine appropriate articulation
                modified_duration = duration  # Default to original duration

                if i % 4 == 0:  # First beat of measure (assuming 4/4)
                    articulation = "accent"
                elif duration >= 1.0:  # Longer notes
                    articulation = "legato"
                elif random.random() < 0.3:  # Some shorter notes are staccato
                    articulation = "staccato"
                    modified_duration = duration * 0.5  # Shorten staccato notes
                else:
                    articulation = "normal"

                result.append((note, modified_duration, volume, articulation))

        else:  # default
            for note, duration, volume in melody:
                modified_duration = duration  # Use original duration
                result.append((note, modified_duration, volume, "normal"))

        return result

    def save_expressive_melody_to_midi(
        self,
        melody: List[Tuple[Optional[str], float, int, str]],
        file_path: str = "expressive_melody.mid",
        instrument: int = 0,
        tempo: int = 120
    ) -> str:
        """Save a melody with dynamics and articulations to MIDI.

        Args:
            melody: List of (note, duration, volume, articulation) tuples
            file_path: Output MIDI file path
            instrument: MIDI program number
            tempo: Tempo in BPM

        Returns:
            Path to the created MIDI file
        """
        midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)
        inst = pretty_midi.Instrument(program=instrument)

        t = 0.0
        for note_data in melody:
            # Unpack note data
            if len(note_data) == 4:
                note_str, duration, velocity, articulation = note_data
            else:
                # Handle case where articulation is not provided
                note_str, duration, velocity = note_data
                articulation = "normal"

            # Skip rests
            if note_str is None:
                t += duration
                continue

            # Parse the note
            name, accidental, octave = note_str[0], 0, 4
            if len(note_str) > 2 and note_str[1] in {'#', 'b'}:
                accidental = 1 if note_str[1] == '#' else -1
                octave = int(note_str[2:]) if note_str[2:].isdigit() else octave
            elif len(note_str) > 1 and note_str[1].isdigit():
                octave = int(note_str[1:])

            semitone = {"C": 0, "D": 2, "E": 4, "F": 5, "G": 7, "A": 9, "B": 11}[name] + accidental
            pitch = 60 + semitone + (octave - 4) * 12

            # Apply articulations
            note_start = t
            note_end = t + duration

            if articulation == "staccato":
                # Make staccato notes shorter
                note_end = t + (duration * 0.5)
            elif articulation == "accent":
                # Accented notes - increase velocity
                velocity = min(127, int(velocity * 1.2))
            # Legato is handled by the duration already

            # Create Note object with the calculated parameters
            midi_note = pretty_midi.Note(
                velocity=velocity,
                pitch=pitch,
                start=note_start,
                end=note_end
            )

            # Add note to instrument
            inst.notes.append(midi_note)

            # Move time forward
            t += duration

        # Add instrument to MIDI file
        midi.instruments.append(inst)
        midi.write(file_path)
        return file_path

    # Variation and form generation

    def create_variation(
        self,
        theme: List[NoteDur],
        variation_type: str = "rhythmic",
        intensity: float = 0.5,
        key: str = "C",
        scale: str = "major"
    ) -> List[NoteDur]:
        """Create a variation on a melodic theme.

        Args:
            theme: Original theme as a list of (note, duration) pairs
            variation_type: Type of variation to create
            intensity: Intensity of the variation (0.0-1.0)
            key: Key name
            scale: Scale name

        Returns:
            Variation as a list of (note, duration) pairs
        """
        result = []

        if variation_type == "rhythmic":
            # Change the rhythm while keeping the pitches
            for note, dur in theme:
                if note is None or random.random() > intensity:
                    result.append((note, dur))
                else:
                    if intensity < 0.3:
                        opts = [dur * 0.5, dur * 2]
                    elif intensity < 0.7:
                        opts = [dur * 0.5, dur * 0.75, dur * 1.5, dur * 2]
                    else:
                        opts = [dur * 0.25, dur * 0.5, dur * 0.75,
                                dur * 1.5, dur * 2, dur * 0.33, dur * 0.66]
                    result.append((note, random.choice(opts)))

        elif variation_type == "melodic":
            # Change the pitches while keeping the rhythm
            for note, dur in theme:
                if note is None or random.random() > intensity:
                    result.append((note, dur))
                else:
                    if intensity < 0.3:
                        ints = [-1, 1, 0, 0]
                    elif intensity < 0.7:
                        ints = [-2, -1, 1, 2]
                    else:
                        ints = [-4, -3, -2, -1, 1, 2, 3, 4]
                    interval = random.choice(ints)
                    if interval != 0:
                        try:
                            result.append((self.transpose(note, interval), dur))
                        except Exception:
                            result.append((note, dur))
                    else:
                        result.append((note, dur))

        elif variation_type == "development":
            # Apply both melodic and rhythmic variations, plus embellishments
            # First, apply melodic and rhythmic variations
            tmp = self.create_variation(theme, "melodic", intensity * 0.7, key, scale)
            tmp = self.create_variation(tmp, "rhythmic", intensity * 0.5, key, scale)
            result = list(tmp)

            # Add embellishments
            i = 0
            while i < len(result) - 1:
                curr_n, curr_d = result[i]
                if curr_n and random.random() < intensity * 0.4:
                    emb_type = random.choice(["passing", "neighbor", "appogiatura"])

                    if emb_type == "passing" and i < len(result) - 1:
                        # Add passing tones between notes
                        nxt, _ = result[i+1]
                        if nxt:
                            try:
                                cm = self.note_to_midi(curr_n)
                                nm = self.note_to_midi(nxt)
                                if abs(nm - cm) >= 2:
                                    direction = 1 if nm > cm else -1
                                    pass_note = self.transpose(curr_n, direction)
                                    result[i] = (curr_n, curr_d/2)
                                    result.insert(i+1, (pass_note, curr_d/2))
                                    i += 1
                            except Exception:
                                pass

                    elif emb_type == "neighbor":
                        # Add neighbor tones
                        try:
                            direction = random.choice([-1, 1])
                            neighbor = self.transpose(curr_n, direction)
                            if curr_d >= 0.75:
                                result[i] = (curr_n, curr_d/3)
                                result.insert(i+1, (neighbor, curr_d/3))
                                result.insert(i+2, (curr_n, curr_d/3))
                                i += 1
                        except Exception:
                            pass

                    elif emb_type == "appogiatura":
                        # Add appogiaturas (grace notes)
                        try:
                            direction = random.choice([-2, -1, 1, 2])
                            app = self.transpose(curr_n, direction)
                            result.insert(i, (app, 0.25))
                            result[i+1] = (curr_n, max(0.5, curr_d - 0.25))
                            i += 1
                        except Exception:
                            pass
                i += 1

        elif variation_type == "inversion":
            # Invert the melody around the tonic
            if not theme:
                return []

            root = {
                "C":60,"C#":61,"Db":61,"D":62,"D#":63,"Eb":63,
                "E":64,"F":65,"F#":66,"Gb":66,"G":67,"G#":68,
                "Ab":68,"A":69,"A#":70,"Bb":70,"B":71
            }[key]

            for note, dur in theme:
                if note is None:
                    result.append((None, dur))
                else:
                    try:
                        mv = self.note_to_midi(note)
                        inv = root - (mv - root)
                        names = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]
                        octv = inv//12 - 1
                        idx = inv % 12
                        result.append((f"{names[idx]}{octv}", dur))
                    except Exception:
                        result.append((note, dur))

        elif variation_type == "retrograde":
            # Play the melody backwards
            result = list(theme)[::-1]

        elif variation_type == "ornamentation":
            # Add baroque-style ornaments
            for note, dur in theme:
                if note is None:
                    result.append((None, dur))
                    continue

                if random.random() < intensity and dur >= 0.75:
                    orn_type = random.choice(["trill", "turn", "mordent", "arpeggio"])

                    if orn_type == "trill" and dur >= 1.0:
                        try:
                            upper_note = self.transpose(note, 1)
                            count = int(dur * 4)
                            sub_dur = dur / count
                            for j in range(count):
                                result.append((note if j%2==0 else upper_note, sub_dur))
                        except Exception:
                            result.append((note, dur))

                    elif orn_type == "turn" and dur >= 1.0:
                        try:
                            upper = self.transpose(note, 1)
                            lower = self.transpose(note, -1)
                            turn_dur = dur/5
                            seq = [note, upper, note, lower, note]
                            for n in seq:
                                result.append((n, turn_dur))
                        except Exception:
                            result.append((note, dur))

                    elif orn_type == "mordent":
                        try:
                            lower = self.transpose(note, -1)
                            result += [(note, dur*0.25), (lower, dur*0.25), (note, dur*0.5)]
                        except Exception:
                            result.append((note, dur))

                    elif orn_type == "arpeggio" and dur >= 1.0:
                        try:
                            if scale == "minor":
                                chord = [note, self.transpose(note, 3), self.transpose(note, 7)]
                            else:
                                chord = [note, self.transpose(note, 4), self.transpose(note, 7)]
                            arp_dur = dur/len(chord)
                            for c in chord:
                                result.append((c, arp_dur))
                        except Exception:
                            result.append((note, dur))
                else:
                    result.append((note, dur))

        else:
            # Default: return the original theme
            result = list(theme)

        # Ensure the total duration of the variation matches the original
        original_duration = sum(d for _, d in theme)
        variation_duration = sum(d for _, d in result)

        # Adjust the last note's duration if needed
        if abs(original_duration - variation_duration) > 0.5 and result:
            last_note, last_dur = result[-1]
            adjusted_dur = last_dur + (original_duration - variation_duration)
            if adjusted_dur > 0:  # Ensure positive duration
                result[-1] = (last_note, adjusted_dur)

        return result

    def create_theme_and_variations(
        self,
        theme_length: int = 8,
        num_variations: int = 3,
        key: str = "C",
        scale: str = "major",
        complexity: float = 0.5
    ) -> List[List[NoteDur]]:
        """Generate a theme and several variations.

        Args:
            theme_length: Length of theme in beats
            num_variations: Number of variations to create
            key: Key name
            scale: Scale name
            complexity: Complexity factor (0.0-1.0)

        Returns:
            List of [theme, var1, var2, ...] where each is a list of (note, duration) pairs
        """
        # Generate the main theme
        theme = self.generate_standalone_melody(
            length=theme_length,
            scale=scale,
            key=key,
            complexity=complexity
        )

        # List of variation types
        variation_types = [
            "rhythmic", "melodic", "development",
            "ornamentation", "inversion", "retrograde"
        ]

        # Start with the theme
        variations = [theme]

        # Generate variations
        for i in range(num_variations):
            # Select variation type
            if i == 0:
                var_type = "melodic"  # First variation is always melodic
            elif i == num_variations - 1:
                var_type = "development"  # Last variation is always development
            else:
                # Avoid repeating the same variation type consecutively
                previous_type = variation_types.index(var_type) if 'var_type' in locals() else -1
                available_types = [t for t in variation_types if variation_types.index(t) != previous_type]
                var_type = random.choice(available_types)

            # Increase intensity with each variation
            intensity = 0.3 + (i / num_variations) * 0.5

            # Create the variation
            variation = self.create_variation(
                theme=theme,
                variation_type=var_type,
                intensity=intensity,
                key=key,
                scale=scale
            )

            variations.append(variation)

        return variations

    def create_sonata_form(
        self,
        key: str = "C",
        scale: str = "major",
        complexity: float = 0.5,
        development_intensity: float = 0.7,
        recapitulation_variation: str = "melodic"
    ) -> Dict[str, List[NoteDur]]:
        """Create a composition in sonata form.

        Args:
            key: Key name
            scale: Scale name
            complexity: Complexity factor (0.0-1.0)
            development_intensity: Intensity of development section variations
            recapitulation_variation: Type of variation for recapitulation

        Returns:
            Dictionary with sections: exposition, development, recapitulation, coda
        """
        # Generate first subject (in tonic key)
        first_subject = self.generate_standalone_melody(8, scale, key, complexity)

        # Determine second subject key (dominant or relative major)
        if scale == "major":
            second_key_note = self.transpose(key+"4", 7)[0]  # Dominant
        else:
            second_key_note = self.transpose(key+"4", 3)[0]  # Relative major

        second_key = second_key_note[:-1] if second_key_note[-1].isdigit() else second_key_note

        # Generate second subject (in dominant or relative major)
        second_subject = self.generate_standalone_melody(8, "major", second_key, complexity*0.8)

        # Create transition between subjects
        transition = self.create_variation(first_subject[-4:], "development", 0.5, key, scale)

        # Exposition: first subject + transition + second subject
        exposition = first_subject + transition + second_subject

        # Development: work with motifs from both subjects
        development = []
        # First subject development
        first_dev = self.create_variation(
            first_subject, "development", development_intensity, key, scale
        )
        development += first_dev[:len(first_dev)//2]

        # Second subject development
        second_dev = self.create_variation(
            second_subject, "development", development_intensity, second_key, "major"
        )
        development += second_dev[:len(second_dev)//2]

        # Return to first subject material to prepare for recapitulation
        return_dev = self.create_variation(first_subject, "rhythmic", development_intensity, key, scale)
        development += return_dev[-4:]

        # Recapitulation: both subjects now in tonic key
        recapitulation = []

        # First subject with ornaments
        first_recap = self.create_variation(first_subject, "ornamentation", 0.3, key, scale)
        recapitulation += first_recap

        # Modified transition
        transition_recap = self.create_variation(transition, "melodic", 0.4, key, scale)
        recapitulation += transition_recap

        # Second subject now in tonic key
        second_recap = self.transpose_melody_to_key(second_subject, second_key, key)
        second_recap = self.create_variation(
            second_recap, recapitulation_variation, 0.3, key, scale
        )
        recapitulation += second_recap

        # Coda from first subject material
        coda = self.create_variation(first_subject[:4], "development", 0.5, key, scale)

        return {
            "exposition": exposition,
            "development": development,
            "recapitulation": recapitulation,
            "coda": coda
        }

    def transpose_melody_to_key(
        self,
        melody: List[NoteDur],
        from_key: str,
        to_key: str
    ) -> List[NoteDur]:
        """Transpose a melody from one key to another.

        Args:
            melody: Melody as a list of (note, duration) pairs
            from_key: Source key name
            to_key: Target key name

        Returns:
            Transposed melody
        """
        mapping = {
            "C":0, "C#":1, "Db":1, "D":2, "D#":3, "Eb":3,
            "E":4, "F":5, "F#":6, "Gb":6, "G":7, "G#":8,
            "Ab":8, "A":9, "A#":10, "Bb":10, "B":11
        }

        # Calculate interval between keys
        shift = mapping.get(to_key, 0) - mapping.get(from_key, 0)

        result = []
        for note, dur in melody:
            if note is None:
                result.append((None, dur))
            else:
                try:
                    result.append((self.transpose(note, shift), dur))
                except Exception:
                    result.append((note, dur))

        return result

    # Percussion and combined output

    def add_percussion(
        self,
        length: int = 4,  # Length in measures
        style: str = "basic",
        complexity: float = 0.5,
        tempo: int = 120
    ) -> Dict[str, List[Tuple[Optional[int], float]]]:
        """Add percussion patterns to a composition.

        Args:
            length: Length in measures
            style: Percussion style ("basic", "rock", "jazz", etc.)
            complexity: Complexity level (0.0-1.0)
            tempo: Tempo in BPM

        Returns:
            Dictionary with percussion parts as (MIDI note, duration) pairs
        """
        # Standard MIDI drum map
        DRUM_MAP = {
            "kick": 36,
            "snare": 38,
            "closed_hi_hat": 42,
            "open_hi_hat": 46,
            "crash": 49,
            "ride": 51,
            "tom_high": 50,
            "tom_mid": 47,
            "tom_low": 45,
            "clap": 39,
            "cowbell": 56
        }

        # Create percussion parts dictionary
        perc_parts = {}

        if style == "basic":
            # Simple kick-snare pattern
            kicks = []
            snares = []
            hi_hats = []

            for measure in range(length):
                # Basic pattern: kick on 1 and 3, snare on 2 and 4
                for beat in range(4):
                    if beat % 2 == 0:  # Beats 1 and 3
                        kicks.append((DRUM_MAP["kick"], 1.0))
                    else:  # Beats 2 and 4
                        snares.append((DRUM_MAP["snare"], 1.0))

                    # Eighth-note hi-hats
                    hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))
                    hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))

            perc_parts["kick"] = kicks
            perc_parts["snare"] = snares
            perc_parts["hi_hat"] = hi_hats

        elif style == "rock":
            # Rock pattern with some variations
            kicks = []
            snares = []
            hi_hats = []
            crashes = []

            for measure in range(length):
                # Common rock pattern
                for beat in range(4):
                    # Kick pattern (1, 3 and variations)
                    if beat == 0 or beat == 2:
                        kicks.append((DRUM_MAP["kick"], 1.0))
                    elif complexity > 0.6 and random.random() < 0.3:
                        # Extra kicks for complexity
                        if random.random() < 0.5:
                            kicks.append((DRUM_MAP["kick"], 0.5))
                            kicks.append((None, 0.5))
                        else:
                            kicks.append((None, 0.5))
                            kicks.append((DRUM_MAP["kick"], 0.5))
                    else:
                        kicks.append((None, 1.0))

                    # Snare on 2 and 4
                    if beat == 1 or beat == 3:
                        snares.append((DRUM_MAP["snare"], 1.0))
                    else:
                        snares.append((None, 1.0))

                    # Eighth-note hi-hats with accents
                    hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))
                    if random.random() < 0.2 and complexity > 0.4:
                        hi_hats.append((DRUM_MAP["open_hi_hat"], 0.5))
                    else:
                        hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))

                # Add crash at start and occasionally elsewhere
                if measure == 0:
                    crashes.append((DRUM_MAP["crash"], 4.0))
                elif random.random() < 0.2 and complexity > 0.5:
                    crashes.append((DRUM_MAP["crash"], 4.0))
                else:
                    crashes.append((None, 4.0))

            perc_parts["kick"] = kicks
            perc_parts["snare"] = snares
            perc_parts["hi_hat"] = hi_hats
            perc_parts["crash"] = crashes

        elif style == "jazz":
            # Swing rhythm with brushes on snare
            kicks = []
            snares = []
            hi_hats = []
            rides = []

            for measure in range(length):
                for beat in range(4):
                    # Kick on 1 and 3
                    if beat % 2 == 0:
                        kicks.append((DRUM_MAP["kick"], 1.0))
                    else:
                        kicks.append((None, 1.0))

                    # Snare with swing feel
                    if beat == 1 or beat == 3:
                        snares.append((DRUM_MAP["snare"], 0.75))
                    else:
                        snares.append((None, 0.75))

                    # Hi-hat on off-beats
                    if beat % 2 == 1:
                        hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))
                    else:
                        hi_hats.append((None, 0.5))

                    # Ride cymbal on 4
                    if beat == 3:
                        rides.append((DRUM_MAP["ride"], 1.0))
                    else:
                        rides.append((None, 1.0))

            perc_parts["kick"] = kicks
            perc_parts["snare"] = snares
            perc_parts["hi_hat"] = hi_hats
            perc_parts["ride"] = rides

        elif style == "latin":
            # Latin percussion pattern
            congas = []
            claves = []
            bongos = []

            for measure in range(length):
                # Clave pattern (3-2 or 2-3)
                pattern = [1, 0, 1, 0, 0, 1, 0, 1, 0, 0]  # 3-2 clave
                for beat in range(4):
                    for eighth in range(2):
                        idx = (beat * 2 + eighth) % len(pattern)
                        if pattern[idx]:
                            claves.append((DRUM_MAP["cowbell"], 0.5))
                        else:
                            claves.append((None, 0.5))

                # Conga pattern
                for beat in range(4):
                    if beat == 0:
                        congas.append((DRUM_MAP["tom_low"], 1.0))  # Low conga
                    elif beat == 2:
                        congas.append((DRUM_MAP["tom_mid"], 1.0))  # Mid conga
                    else:
                        if random.random() < complexity:
                            congas.append((DRUM_MAP["tom_high"], 0.5))  # High conga
                            congas.append((None, 0.5))
                        else:
                            congas.append((DRUM_MAP["tom_high"], 1.0))  # High conga

                # Bongo pattern with variations
                for beat in range(4):
                    for eighth in range(2):
                        if random.random() < 0.6:
                            bongos.append((DRUM_MAP["tom_high"], 0.5))
                        else:
                            bongos.append((None, 0.5))
                        if random.random() < 0.4:
                            bongos.append((DRUM_MAP["tom_high"], 0.5))
                        else:
                            bongos.append((None, 0.5))
                        if random.random() < 0.9:
                            bongos.append((DRUM_MAP["tom_high"], 0.5))
                        else:
                            bongos.append((None, 0.5))
                        if random.random() < 0.3:
                            bongos.append((DRUM_MAP["tom_high"], 0.5))
                        else:
                            bongos.append((None, 0.5))

            perc_parts["claves"] = claves
            perc_parts["congas"] = congas
            perc_parts["bongos"] = bongos

        elif style == "electronic":
            # Electronic music pattern
            kicks = []
            snares = []
            hi_hats = []

            for measure in range(length):
                # Four-on-the-floor kick
                for beat in range(4):
                    kicks.append((DRUM_MAP["kick"], 1.0))

                # Offbeat hi-hats (eighths)
                for beat in range(4):
                    for eighth in range(2):
                        if eighth == 1:  # Offbeat
                            hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.5))
                        else:
                            hi_hats.append((None, 0.5))

                # Snare on 2 and 4
                for beat in range(4):
                    if beat == 1 or beat == 3:
                        snares.append((DRUM_MAP["snare"], 1.0))
                    else:
                        snares.append((None, 1.0))

            # Add complexity with additional hi-hat patterns
            if complexity > 0.5:
                hi_hats = []
                for measure in range(length):
                    for beat in range(4):
                        if random.random() < 0.8:
                            hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.25))
                        else:
                            hi_hats.append((None, 0.25))
                        if random.random() < 0.4:
                            hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.25))
                        else:
                            hi_hats.append((None, 0.25))
                        if random.random() < 0.9:
                            hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.25))
                        else:
                            hi_hats.append((None, 0.25))
                        if random.random() < 0.3:
                            hi_hats.append((DRUM_MAP["closed_hi_hat"], 0.25))
                        else:
                            hi_hats.append((None, 0.25))

            perc_parts["kick"] = kicks
            perc_parts["snare"] = snares
            perc_parts["hi_hat"] = hi_hats

        return perc_parts

    def combine_melody_and_percussion(
        self,
        melody_parts: Dict[str, List[NoteDur]],
        percussion_parts: Dict[str, List[Tuple[Optional[int], float]]],
        file_path: str = "composition.mid",
        tempo: int = 120
    ) -> str:
        """Combine melodic parts and percussion into a single MIDI file.

        Args:
            melody_parts: Dictionary of melodic parts (instrument: [(note, duration), ...])
            percussion_parts: Dictionary of percussion parts (drum: [(note, duration), ...])
            file_path: Output file path
            tempo: Tempo in BPM

        Returns:
            Path to the saved MIDI file
        """
        midi = pretty_midi.PrettyMIDI(initial_tempo=tempo)

        # Instrument MIDI program mapping
        instrument_mapping = {
            'Piano': 0, 'Violin': 40, 'Flute': 73, 'Clarinet': 71,
            'Trumpet': 56, 'Guitar': 24, 'Music Box': 10, 'Harp': 46,
            'Bass': 32, 'Cello': 42
        }

        # Add melodic instruments
        for instrument_name, notes in melody_parts.items():
            program = instrument_mapping.get(instrument_name, 0)

            # Create MIDI instrument
            midi_instrument = pretty_midi.Instrument(program=program, name=instrument_name)

            # Add notes
            time = 0.0
            for note_str, duration in notes:
                # Skip rests
                if note_str is None:
                    time += duration
                    continue

                # Parse the note
                if len(note_str) > 1:
                    try:
                        note_name = note_str[0]

                        # Handle accidentals
                        if len(note_str) > 2 and note_str[1] in ['#', 'b']:
                            accidental = 1 if note_str[1] == '#' else -1
                            octave_pos = 2
                        else:
                            accidental = 0
                            octave_pos = 1

                        if len(note_str) > octave_pos and note_str[octave_pos].isdigit():
                            octave = int(note_str[octave_pos:])
                        else:
                            octave = 4  # Default octave

                        # Map note name to semitone offset from C
                        note_map = {'C': 0, 'D': 2, 'E': 4, 'F': 5, 'G': 7, 'A': 9, 'B': 11}
                        semitone = note_map.get(note_name, 0) + accidental

                        # Calculate MIDI note number
                        midi_note = 60 + semitone + (octave - 4) * 12

                        # Create Note object
                        note = pretty_midi.Note(
                            velocity=100,
                            pitch=midi_note,
                            start=time,
                            end=time + duration
                        )

                        # Add note to instrument
                        midi_instrument.notes.append(note)
                    except Exception:
                        # Skip notes that can't be parsed
                        pass

                time += duration

            # Add instrument to MIDI file
            midi.instruments.append(midi_instrument)

        # Add percussion track (if any)
        if percussion_parts:
            drum_track = pretty_midi.Instrument(program=0, is_drum=True, name="Drums")

            # Process each percussion part
            for drum_name, notes in percussion_parts.items():
                time = 0.0
                for note_num, duration in notes:
                    if note_num is not None:
                        # Create drum hit as a Note object
                        note = pretty_midi.Note(
                            velocity=100,
                            pitch=note_num,  # MIDI note number for this drum
                            start=time,
                            end=time + min(duration, 0.1)  # Drums are short
                        )
                        drum_track.notes.append(note)

                    time += duration

            # Add drum track to MIDI file
            midi.instruments.append(drum_track)

        # Write the MIDI file
        midi.write(file_path)
        return file_path

# UI component setup - only if running in an environment with widgets
if _HAS_WIDGETS:
    # Initialize the generator instance
    generator = OrchestraMarkovGenerator()

    # Create output widgets
    output = Output()
    melody_output = Output()
    ensemble_output = Output()
    advanced_composition_output = Output()

    # Create UI control elements

    # Scale and key controls
    scale_dropdown = Dropdown(
        options=['major', 'minor', 'pentatonic', 'blues', 'chromatic'],
        value='major',
        description='Scale:',
    )

    key_dropdown = Dropdown(
        options=['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B'],
        value='C',
        description='Key:',
    )

    # Length controls
    melody_length_slider = IntSlider(
        value=16,
        min=8,
        max=32,
        step=4,
        description='Length (beats):',
        continuous_update=False,
    )

    composition_length_slider = IntSlider(
        value=16,
        min=8,
        max=64,
        step=4,
        description='Length (beats):',
        continuous_update=False,
    )

    # Complexity and pattern controls
    melody_complexity_slider = FloatSlider(
        value=0.5,
        min=0.1,
        max=1.0,
        step=0.1,
        description='Complexity:',
        continuous_update=False,
        tooltip='Higher values create more complex melodies with larger intervals'
    )

    melody_markov_slider = IntSlider(
        value=1,
        min=1,
        max=3,
        step=1,
        description='Pattern Memory:',
        continuous_update=False,
        tooltip='Higher values create more structured melodic patterns'
    )

    # Tempo control
    tempo_slider = IntSlider(
        value=120,
        min=60,
        max=180,
        step=5,
        description='Tempo (BPM):',
        continuous_update=False,
    )

    # Instrument selection for melody
    melody_instrument_dropdown = Dropdown(
        options=['Piano', 'Violin', 'Flute', 'Clarinet', 'Trumpet', 'Guitar', 'Music Box', 'Harp'],
        value='Piano',
        description='Instrument:',
    )

    # Form dropdown
    form_dropdown = Dropdown(
        options=['Theme and Variations', 'Sonata Form', 'Free Form'],
        value='Theme and Variations',
        description='Form:',
    )

    # Percussion style dropdown
    percussion_style_dropdown = Dropdown(
        options=['None', 'basic', 'rock', 'jazz', 'latin', 'electronic'],
        value='None',
        description='Percussion:',
    )

    # Dynamics and articulation dropdowns
    dynamics_dropdown = Dropdown(
        options=['none', 'arch', 'wave', 'terraced', 'random'],
        value='arch',
        description='Dynamics:',
    )

    articulation_dropdown = Dropdown(
        options=['normal', 'legato', 'staccato', 'mixed', 'phrase'],
        value='mixed',
        description='Articulation:',
    )

    # Define instrument options with their roles
    instruments = [
        {'name': 'Piano', 'program': 0, 'roles': ['melody', 'harmony', 'bass']},
        {'name': 'Violin', 'program': 40, 'roles': ['melody', 'harmony']},
        {'name': 'Flute', 'program': 73, 'roles': ['melody', 'harmony']},
        {'name': 'Clarinet', 'program': 71, 'roles': ['melody', 'harmony']},
        {'name': 'Trumpet', 'program': 56, 'roles': ['melody', 'harmony']},
        {'name': 'Guitar', 'program': 24, 'roles': ['melody', 'harmony', 'bass']},
        {'name': 'Bass', 'program': 32, 'roles': ['bass']},
        {'name': 'Cello', 'program': 42, 'roles': ['harmony', 'bass']},
        {'name': 'Music Box', 'program': 10, 'roles': ['melody']},
        {'name': 'Harp', 'program': 46, 'roles': ['melody', 'harmony']}
    ]

    # Create checkboxes for each instrument
    instrument_checkboxes = {}
    role_dropdowns = {}

    # Create UI for instrument selection
    instrument_selection_ui = []

    for instrument in instruments:
        # Create checkbox for this instrument
        checkbox = Checkbox(
            value=False,
            description=instrument['name'],
            disabled=False,
            indent=False
        )
        instrument_checkboxes[instrument['name']] = checkbox

        # Create role dropdown for this instrument
        dropdown = Dropdown(
            options=instrument['roles'],
            value=instrument['roles'][0],  # Default to first available role
            description='Role:',
            disabled=True,  # Initially disabled until checkbox is checked
            layout={'width': '150px'}
        )
        role_dropdowns[instrument['name']] = dropdown

        # Connect checkbox to enable/disable dropdown
        def make_checkbox_handler(name):
            def handle_checkbox_change(change):
                role_dropdowns[name].disabled = not change.new
            return handle_checkbox_change

        checkbox.observe(make_checkbox_handler(instrument['name']), names='value')

        # Add a row for this instrument with checkbox and dropdown
        instrument_selection_ui.append(HBox([checkbox, dropdown]))

    # Function to get selected instruments and their roles
    def get_selected_instruments():
        selected = []
        roles = []

        for name, checkbox in instrument_checkboxes.items():
            if checkbox.value:
                selected.append(name)
                roles.append(role_dropdowns[name].value)

        return selected, roles

    # Core generation functions for UI interaction

    def generate_melody(b=None):
        """Generate a standalone melody with UI output"""
        try:
            # Get values from melody control widgets
            length = melody_length_slider.value
            scale = scale_dropdown.value
            key = key_dropdown.value
            complexity = melody_complexity_slider.value
            instrument = melody_instrument_dropdown.value
            markov_order = melody_markov_slider.value
            tempo = tempo_slider.value

            # Generate the melody
            melody = generator.generate_standalone_melody(
                length=length,
                scale=scale,
                key=key,
                complexity=complexity,
                markov_order=markov_order
            )

            # Save to MIDI
            midi_program = {
                'Piano': 0,
                'Violin': 40,
                'Flute': 73,
                'Clarinet': 71,
                'Trumpet': 56,
                'Guitar': 24,
                'Music Box': 10,
                'Harp': 46
            }.get(instrument, 0)

            midi_file = generator.save_melody_to_midi(
                melody=melody,
                file_path="/content/generated_melody.mid",
                instrument=midi_program,
                tempo=tempo
            )

            # Create text output
            output_text = f"Generated Melody\n\n"
            output_text += f"Scale: {scale} in the key of {key}\n"
            output_text += f"Length: {length} beats\n"
            output_text += f"Complexity: {complexity}\n"
            output_text += f"Instrument: {instrument} (MIDI Program {midi_program})\n"
            output_text += f"Tempo: {tempo} BPM\n\n"

            output_text += "Melody Notes:\n"
            total_duration = 0
            note_count = 0
            for note, duration in melody:
                if note is not None:  # Skip rests
                    output_text += f"  {note} ({duration} beats)\n"
                    total_duration += duration
                    note_count += 1

            output_text += f"\nTotal: {note_count} notes spanning {total_duration} beats"

            # Save text output
            text_output_file = "/content/generated_melody.txt"
            with open(text_output_file, 'w') as f:
                f.write(output_text)

            # Convert MIDI to WAV
            audio_available = False
            wav_file = "/content/generated_melody.wav"
            soundfont_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"  # Default soundfont location

            # Check if soundfont exists, if not, try to download it
            if not os.path.exists(soundfont_path):
                try:
                    subprocess.run(['wget', '-q', 'https://github.com/abdelouahabb/sf2-host/raw/main/FluidR3_GM.sf2', '-O', '/content/FluidR3_GM.sf2'], check=True)
                    soundfont_path = "/content/FluidR3_GM.sf2"
                except Exception:
                    print("Could not download soundfont")

            # Try to convert MIDI to WAV
            try:
                if os.path.exists(soundfont_path) and os.path.exists(midi_file):
                    result = subprocess.run([
                        "fluidsynth", "-ni", soundfont_path,
                        midi_file, "-F", wav_file, "-r", "44100"
                    ], capture_output=True, text=True)

                    audio_available = os.path.exists(wav_file)
            except Exception as e:
                print(f"Error converting MIDI to audio: {str(e)}")
                audio_available = False

            # Display results
            with melody_output:
                clear_output()
                print("🎵 Generated Melody")
                print(output_text)
                print(f"\nMIDI file saved to: {midi_file}")

                # Display audio if available
                if audio_available:
                    print(f"Audio file saved to: {wav_file}")
                    display(Audio(wav_file, rate=44100))

                # Download files
                try:
                    from google.colab import files
                    print("\nDownloading files...")
                    files.download(text_output_file)
                    files.download(midi_file)
                    if audio_available:
                        files.download(wav_file)
                    print("Files downloaded successfully!")
                except Exception as e:
                    print(f"Error downloading files: {str(e)}")

            return melody
        except Exception as e:
            with melody_output:
                clear_output()
                print(f"Error generating melody: {str(e)}")
                traceback.print_exc()
                return None


    def generate_ensemble(b=None):
        """Generate and play a multi-instrument composition"""
        try:
            # Get values from widgets
            length = melody_length_slider.value
            scale = scale_dropdown.value
            key = key_dropdown.value
            complexity = melody_complexity_slider.value
            tempo = tempo_slider.value
            tempo_slider.value

            # Get selected instruments and their roles
            instruments, techniques = get_selected_instruments()

            if not instruments:
                with ensemble_output:
                    clear_output()
                    print("Please select at least one instrument")
                    return None

            # Generate the composition
            composition = generator.generate_multi_instrument_composition(
                length=length,
                scale=scale,
                key=key,
                instruments=instruments,
                techniques=techniques,
                tempo=tempo,
                complexity=complexity
            )

            # Save to MIDI
            midi_file = generator.save_multi_instrument_to_midi(
                composition=composition,
                file_path="/content/ensemble_composition.mid",
                tempo=tempo
            )

            # Create output text
            output_text = f"Generated Multi-Instrument Composition\n\n"
            output_text += f"Scale: {scale} in the key of {key}\n"
            output_text += f"Length: {length} beats\n"
            output_text += f"Complexity: {complexity}\n"
            output_text += f"Tempo: {tempo} BPM\n\n"

            output_text += "Instrumentation:\n"
            for instrument, role in zip(instruments, techniques):
                output_text += f"  {instrument} ({role}): {len(composition[instrument])} notes\n"

            # Save text output
            text_output_file = "/content/ensemble_composition.txt"
            with open(text_output_file, 'w') as f:
                f.write(output_text)

            # Convert MIDI to WAV
            audio_available = False
            wav_file = "/content/ensemble_composition.wav"
            soundfont_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"  # Default soundfont location

            # Check if soundfont exists, if not, try to download it
            if not os.path.exists(soundfont_path):
                try:
                    subprocess.run(['wget', '-q', 'https://github.com/abdelouahabb/sf2-host/raw/main/FluidR3_GM.sf2', '-O', '/content/FluidR3_GM.sf2'], check=True)
                    soundfont_path = "/content/FluidR3_GM.sf2"
                except Exception:
                    print("Could not download soundfont")

            # Try to convert MIDI to WAV
            try:
                if os.path.exists(soundfont_path) and os.path.exists(midi_file):
                    result = subprocess.run([
                        "fluidsynth", "-ni", soundfont_path,
                        midi_file, "-F", wav_file, "-r", "44100"
                    ], capture_output=True, text=True)

                    audio_available = os.path.exists(wav_file)
            except Exception as e:
                print(f"Error converting MIDI to audio: {str(e)}")
                audio_available = False

            # Display results
            with ensemble_output:
                clear_output()
                print("🎻🎹🎷 Generated Multi-Instrument Composition")
                print(output_text)
                print(f"\nMIDI file saved to: {midi_file}")

                # Display audio if available
                if audio_available:
                    print(f"Audio file saved to: {wav_file}")
                    display(Audio(wav_file, rate=44100))

                # Download files
                try:
                    from google.colab import files
                    print("\nDownloading files...")
                    files.download(text_output_file)
                    files.download(midi_file)
                    if audio_available:
                        files.download(wav_file)
                    print("Files downloaded successfully!")
                except Exception as e:
                    print(f"Error downloading files: {str(e)}")

            return composition
        except Exception as e:
            with ensemble_output:
                clear_output()
                print(f"Error generating ensemble composition: {str(e)}")
                traceback.print_exc()
                return None

    def generate_advanced_composition(b=None):
        """Generate advanced composition with form, dynamics, and articulation"""
        try:
            # Get values from widgets
            length = composition_length_slider.value
            scale = scale_dropdown.value
            key = key_dropdown.value
            complexity = melody_complexity_slider.value
            tempo = tempo_slider.value
            form = form_dropdown.value
            percussion_style = percussion_style_dropdown.value
            dynamics_shape = dynamics_dropdown.value
            articulation_style = articulation_dropdown.value

            # Get selected instruments and their roles
            selected_instruments, techniques = get_selected_instruments()

            if not selected_instruments:
                with advanced_composition_output:
                    clear_output()
                    print("Please select at least one instrument")
                    return None

            # Generate composition based on form
            if form == "Theme and Variations":
                variations = generator.create_theme_and_variations(
                    theme_length=min(length, 16),  # Reasonable theme length
                    num_variations=max(1, length // 16),  # Number of variations based on total length
                    key=key,
                    scale=scale,
                    complexity=complexity
                )

                # Convert variations to instrument parts
                composition = {}
                for instrument, role in zip(selected_instruments, techniques):
                    if role == "melody":
                        # Assign all variations sequentially
                        melody_part = []
                        for var in variations:
                            melody_part.extend(var)
                        composition[instrument] = melody_part
                    elif role == "harmony":
                        # Create harmony for each variation
                        harmony_part = []
                        for var in variations:
                            # Generate harmony based on the melody
                            current_harmony = []
                            for note, dur in var:
                                if note is not None:
                                    # Add a harmony note (typically a third)
                                    try:
                                        if scale == "minor":
                                            harmony_note = generator.transpose(note, 3)  # Minor third
                                        else:
                                            harmony_note = generator.transpose(note, 4)  # Major third
                                        current_harmony.append((harmony_note, dur))
                                    except Exception:
                                        current_harmony.append((note, dur))
                                else:
                                    current_harmony.append((None, dur))
                            harmony_part.extend(current_harmony)
                        composition[instrument] = harmony_part
                    elif role == "bass":
                        # Create bass line for each variation
                        bass_part = []
                        for var in variations:
                            # Simplified bass line with longer notes
                            current_bass = []
                            current_time = 0
                            while current_time < sum(d for _, d in var):
                                # Add root note in the bass register
                                bass_note = f"{key}2"  # Root note in bass register
                                bass_duration = min(2.0, sum(d for _, d in var) - current_time)
                                current_bass.append((bass_note, bass_duration))
                                current_time += bass_duration
                            bass_part.extend(current_bass)
                        composition[instrument] = bass_part

            elif form == "Sonata Form":
                # Generate sonata form
                sonata_sections = generator.create_sonata_form(
                    key=key,
                    scale=scale,
                    complexity=complexity,
                    development_intensity=0.7,
                    recapitulation_variation="melodic"
                )

                # Convert sonata sections to instrument parts
                composition = {}
                for instrument, role in zip(selected_instruments, techniques):
                    if role == "melody":
                        # Main melody gets all sections
                        melody_part = []
                        for section in ["exposition", "development", "recapitulation", "coda"]:
                            melody_part.extend(sonata_sections[section])
                        composition[instrument] = melody_part
                    elif role == "harmony":
                        # Harmony follows the melody
                        harmony_part = []
                        for section in ["exposition", "development", "recapitulation", "coda"]:
                            # Generate harmony based on melody
                            for note, dur in sonata_sections[section]:
                                if note is not None:
                                    try:
                                        if scale == "minor":
                                            harmony_note = generator.transpose(note, 3)
                                        else:
                                            harmony_note = generator.transpose(note, 4)
                                        harmony_part.append((harmony_note, dur))
                                    except Exception:
                                        harmony_part.append((note, dur))
                                else:
                                    harmony_part.append((None, dur))
                        composition[instrument] = harmony_part
                    elif role == "bass":
                        # Bass line follows harmony but simpler
                        bass_part = []
                        for section in ["exposition", "development", "recapitulation", "coda"]:
                            current_time = 0
                            total_section_duration = sum(d for _, d in sonata_sections[section])
                            while current_time < total_section_duration:
                                # Bass note is usually root or fifth
                                if section == "development":
                                    # More movement in development
                                    notes = [f"{key}2", generator.transpose(f"{key}2", 7)]
                                    bass_note = random.choice(notes)
                                else:
                                    bass_note = f"{key}2"  # Root note in bass register

                                bass_duration = min(2.0, total_section_duration - current_time)
                                bass_part.append((bass_note, bass_duration))
                                current_time += bass_duration
                        composition[instrument] = bass_part

            else:  # Free Form
                # Generate a simpler multi-instrument composition
                composition = generator.generate_multi_instrument_composition(
                    length=length,
                    scale=scale,
                    key=key,
                    instruments=selected_instruments,
                    techniques=techniques,
                    tempo=tempo,
                    complexity=complexity
                )

            # Add percussion if selected
            if percussion_style != "None":
                perc_length = length // 4  # Length in measures
                percussion_parts = generator.add_percussion(
                    length=perc_length,
                    style=percussion_style,
                    complexity=complexity,
                    tempo=tempo
                )
            else:
                percussion_parts = {}

            # Add dynamics and articulations to the first melodic instrument
            if selected_instruments and dynamics_shape != "none":
                # Find a melodic instrument
                melody_instrument = None
                for instrument, role in zip(selected_instruments, techniques):
                    if role == "melody" and instrument in composition:
                        melody_instrument = instrument
                        break

                if melody_instrument:
                    # Add dynamics
                    melodic_part = composition[melody_instrument]
                    with_dynamics = generator.add_dynamics(melodic_part, shape=dynamics_shape)

                    # Add articulations
                    with_articulations = generator.add_articulations(with_dynamics, style=articulation_style)

                    # Look up the instrument program number using the instrument name
                    instrument_program = None
                    # Reference the globally defined instruments list containing dictionaries
                    for instr_dict in instruments:
                        if instr_dict['name'] == melody_instrument:
                            instrument_program = instr_dict['program']
                            break

                    if instrument_program is None:
                        # Use a default program number if the instrument is not found
                        instrument_program = 0

                    # Save expressive version
                    expressive_midi_file = generator.save_expressive_melody_to_midi(
                        melody=with_articulations,
                        file_path="/content/expressive_melody.mid",
                        instrument=instrument_program,
                        tempo=tempo
                    )

            # Combine everything and save to MIDI
            midi_file = generator.combine_melody_and_percussion(
                melody_parts=composition,
                percussion_parts=percussion_parts,
                file_path="/content/advanced_composition.mid",
                tempo=tempo
            )

            # Create output text
            output_text = f"Generated Advanced Composition\n\n"
            output_text += f"Form: {form}\n"
            output_text += f"Scale: {scale} in the key of {key}\n"
            output_text += f"Length: {length} beats\n"
            output_text += f"Complexity: {complexity}\n"
            output_text += f"Tempo: {tempo} BPM\n"
            output_text += f"Percussion: {percussion_style}\n"
            output_text += f"Dynamics: {dynamics_shape}\n"
            output_text += f"Articulation: {articulation_style}\n\n"

            output_text += "Instrumentation:\n"
            for instrument, role in zip(selected_instruments, techniques):
                if instrument in composition:
                    output_text += f"  {instrument} ({role}): {len(composition[instrument])} notes\n"

            # Save text output
            text_output_file = "/content/advanced_composition.txt"
            with open(text_output_file, 'w') as f:
                f.write(output_text)

            # Convert MIDI to WAV
            audio_available = False
            wav_file = "/content/advanced_composition.wav"
            soundfont_path = "/usr/share/sounds/sf2/FluidR3_GM.sf2"

            # Check if soundfont exists, if not, try to download it
            if not os.path.exists(soundfont_path):
                try:
                    subprocess.run(['wget', '-q', 'https://github.com/abdelouahabb/sf2-host/raw/main/FluidR3_GM.sf2', '-O', '/content/FluidR3_GM.sf2'], check=True)
                    soundfont_path = "/content/FluidR3_GM.sf2"
                except Exception:
                    print("Could not download soundfont")

            # Try to convert MIDI to WAV
            try:
                if os.path.exists(soundfont_path) and os.path.exists(midi_file):
                    result = subprocess.run([
                        "fluidsynth", "-ni", soundfont_path,
                        midi_file, "-F", wav_file, "-r", "44100"
                    ], capture_output=True, text=True)

                    audio_available = os.path.exists(wav_file)
            except Exception as e:
                print(f"Error converting MIDI to audio: {str(e)}")
                audio_available = False

            # Display results
            with advanced_composition_output:
                clear_output()
                print("🎼 Generated Advanced Composition")
                print(output_text)
                print(f"\nMIDI file saved to: {midi_file}")

                # Display audio if available
                if audio_available:
                    print(f"Audio file saved to: {wav_file}")
                    display(Audio(wav_file, rate=44100))

                # Download files
                try:
                    from google.colab import files
                    print("\nDownloading files...")
                    files.download(text_output_file)
                    files.download(midi_file)
                    if audio_available:
                        files.download(wav_file)
                    print("Files downloaded successfully!")
                except Exception as e:
                    print(f"Error downloading files: {str(e)}")

            return composition

        except Exception as e:
            with advanced_composition_output:
                clear_output()
                print(f"Error generating advanced composition: {str(e)}")
                traceback.print_exc()
                return None

    # Create the generation buttons
    generate_melody_button = Button(
        description='Generate Melody',
        button_style='info',
        tooltip='Generate a standalone melody'
    )
    generate_melody_button.on_click(lambda b: generate_melody())

    generate_ensemble_button = Button(
        description='Generate Ensemble',
        button_style='success',
        tooltip='Generate a composition with multiple instruments'
    )
    generate_ensemble_button.on_click(lambda b: generate_ensemble())

    generate_advanced_button = Button(
        description='Generate Advanced Composition',
        button_style='danger',
        tooltip='Generate a complex musical composition with form, dynamics, and articulations'
    )
    generate_advanced_button.on_click(lambda b: generate_advanced_composition())

    # Create the UI for melody generation
    melody_controls = VBox([
        HBox([scale_dropdown, key_dropdown]),
        melody_length_slider,
        melody_complexity_slider,
        melody_markov_slider,
        tempo_slider,
        melody_instrument_dropdown,
        generate_melody_button,
        melody_output
    ])

    # Create the ensemble controls
    ensemble_controls = VBox([
        HBox([scale_dropdown, key_dropdown]),
        melody_length_slider,
        melody_complexity_slider,
        tempo_slider,
        HTML("<h3>Select Instruments and Roles</h3>"),
        HTML("<p>Check the instruments you want to include and select their roles in the ensemble.</p>"),
        *instrument_selection_ui,  # Unpack all instrument UI rows
        generate_ensemble_button,
        ensemble_output
    ])

    # Create the advanced composition controls
    advanced_composition_controls = VBox([
        HTML("<h3>Advanced Musical Composition</h3>"),
        HTML("<p>Create complex compositions with musical form, percussion, dynamics, and articulations.</p>"),
        HBox([scale_dropdown, key_dropdown]),
        composition_length_slider,
        melody_complexity_slider,
        tempo_slider,
        HBox([form_dropdown, percussion_style_dropdown]),
        HBox([dynamics_dropdown, articulation_dropdown]),
        HTML("<h3>Select Instruments and Roles</h3>"),
        HTML("<p>Check the instruments you want to include and select their roles in the ensemble.</p>"),
        *instrument_selection_ui,  # Reuse the instrument selection UI
        generate_advanced_button,
        advanced_composition_output
    ])

    # Create basic controls for backward compatibility
    basic_tab = VBox([
        HTML("<h3>Basic Controls</h3>"),
        HTML("<p>Simple controls for quick generation of musical patterns.</p>"),
        HBox([scale_dropdown, key_dropdown]),
        melody_length_slider,
        melody_complexity_slider,
        generate_melody_button,
        melody_output
    ])

    # Create advanced controls for backward compatibility
    advanced_tab = VBox([
        HTML("<h3>Advanced Controls</h3>"),
        HTML("<p>More detailed control over the music generation process.</p>"),
        HBox([scale_dropdown, key_dropdown]),
        melody_length_slider,
        melody_complexity_slider,
        melody_markov_slider,
        tempo_slider,
        generate_ensemble_button,
        ensemble_output
    ])

    # Assemble tabs
    tab = Tab(children=[
        basic_tab,
        advanced_tab,
        melody_controls,
        ensemble_controls,
        advanced_composition_controls
    ])
    tab.set_title(0, 'Basic Controls')
    tab.set_title(1, 'Advanced Controls')
    tab.set_title(2, 'Melody Generator')
    tab.set_title(3, 'Ensemble Generator')
    tab.set_title(4, 'Advanced Composition')

    # Display the UI
    display(VBox([tab, output]))

    # Display initial instructions in the main output area
    with output:
        clear_output()  # Clear previous instructions if any
        print("Welcome to the Orchestra Markov Chain Generator!")
        print("\nThis tool generates orchestral compositions and melodies using Markov chain processes.")
        print("Use the tabs to access different features:")
        print("\nBasic Controls:")
        print("- Scale & Key: Choose the musical scale and key for your composition")
        print("- Length: Set how long your melody or composition will be")
        print("- Complexity: Higher values create more complex patterns")
        print("\nMelody Generator:")
        print("- Generate standalone melodies with customizable parameters")
        print("- Control scale, key, complexity, and instrument")
        print("\nEnsemble Generator:")
        print("- Generate multi-instrument compositions")
        print("- Assign roles like melody, harmony, and bass to instruments")
        print("\nAdvanced Composition:")
        print("- Create complex compositions with formal structure")
        print("- Add percussion, dynamics, and articulations")
        print("- Generate sophisticated musical forms like sonata form")
        print("\nSelect a tab and click the corresponding generate button to create music!")


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\15622\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
